In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from lime import lime_image
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt
from tensorflow.keras.applications.imagenet_utils import decode_predictions

# Define your CNN model
def build_cnn(input_shape, num_classes):
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Load your dataset and preprocess it
# Assuming X_train, y_train, X_test, y_test are loaded and preprocessed

# Define your model parameters
input_shape = X_train.shape[1:]
num_classes = len(np.unique(y_train))

# Build the CNN model
model = build_cnn(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Grad-CAM
def grad_cam(input_model, image, cls, layer_name):
    y_c = input_model.output[0, cls]
    conv_output = input_model.get_layer(layer_name).output
    grads = tf.gradients(y_c, conv_output)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    iterate = tf.keras.backend.function([input_model.input], [pooled_grads, conv_output[0]])
    pooled_grads_value, conv_output_value = iterate(image)
    for i in range(pooled_grads.shape[0]):
        conv_output_value[:, :, i] *= pooled_grads_value[i]
    heatmap = np.mean(conv_output_value, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    return heatmap

# LIME
def lime_explanation(model, image, top_labels=5, hide_color=None):
    explainer = lime_image.LimeImageExplainer()
    explanation = explainer.explain_instance(image[0].astype('double'), model.predict, top_labels=top_labels, hide_color=hide_color)
    return explanation

# Choose an image for interpretation
image_idx = 0
image = X_test[image_idx][np.newaxis, ...]
true_label = y_test[image_idx]

# Grad-CAM interpretation
heatmap = grad_cam(model, image, true_label, 'conv2d_2')  # You may need to adjust the layer name
plt.matshow(heatmap)
plt.title('Grad-CAM')
plt.show()

# LIME interpretation
expl = lime_explanation(model, image)
temp, mask = expl.get_image_and_mask(expl.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))
plt.title('LIME')
plt.show()


ModuleNotFoundError: No module named 'lime'

In [2]:
!pip install lime

^C


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16
from lime import lime_image
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt

# Define your CNN model
def create_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Load and preprocess data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'train_data_dir',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    'validation_data_dir',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

# Create and compile the model
input_shape = (150, 150, 3)
num_classes = len(train_generator.class_indices)
model = create_model(input_shape, num_classes)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples/train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples/validation_generator.batch_size)

# Prediction function
def predict_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(150, 150))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    predicted_label = list(train_generator.class_indices.keys())[predicted_class]

    return predicted_label, prediction

# Grad-CAM
def grad_cam(image_path, layer_name='conv2d_3'):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(150, 150))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    preds = model.predict(img_array)
    class_idx = np.argmax(preds[0])
    class_output = model.output[:, class_idx]
    last_conv_layer = model.get_layer(layer_name)

    grads = tf.GradientTape().gradient(class_output, last_conv_layer.output)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, last_conv_layer.output), axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    img = img_array[0]
    heatmap = tf.image.resize(heatmap, (img.shape[0], img.shape[1]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = tf.expand_dims(heatmap, axis=-1)
    heatmap = tf.concat([heatmap, heatmap, heatmap], axis=-1)
    superimposed_img = tf.cast(img * 255, tf.float32) + heatmap
    superimposed_img = superimposed_img / np.max(superimposed_img)

    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()

# LIME
def lime_explanation(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(150, 150))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    explainer = lime_image.LimeImageExplainer()
    explanation = explainer.explain_instance(img_array[0], model.predict, top_labels=5, hide_color=0, num_samples=1000)

    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=5, hide_rest=False)
    img_boundry1 = mark_boundaries(temp / 2 + 0.5, mask)

    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False)
    img_boundry2 = mark_boundaries(temp / 2 + 0.5, mask)

    plt.imshow(img_boundry1)
    plt.axis('off')
    plt.show()

    plt.imshow(img_boundry2)
    plt.axis('off')
    plt.show()

# Example usage
image_path = 'example_image.jpg'
predicted_label, prediction = predict_image(image_path)
print("Predicted Label:", predicted_label)
print("Prediction:", prediction)

grad_cam(image_path)
lime_explanation(image_path)


ModuleNotFoundError: No module named 'tensorflow'

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/125-136 (1)'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import keras
from keras.models import Sequential
from keras.layers import Conv2D,Flatten,Dense,MaxPooling2D,Dropout
from sklearn.metrics import accuracy_score
from keras.layers import Flatten

import ipywidgets as widgets
import io
from PIL import Image
import tqdm
from sklearn.model_selection import train_test_split
import cv2
from sklearn.utils import shuffle
import tensorflow as tf
from keras import regularizers

X_train = []
Y_train = []
image_size = 150
labels = ['glioma_tumor','meningioma_tumor','no_tumor','pituitary_tumor']
for i in labels:
    folderPath = os.path.join('/content/drive/MyDrive/125-136 (1)/Training',i)
    for j in os.listdir(folderPath):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train.append(img)
        Y_train.append(i)

for i in labels:
    folderPath = os.path.join('/content/drive/MyDrive/125-136 (1)/Testing',i)
    for j in os.listdir(folderPath):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train.append(img)
        Y_train.append(i)

X_train = np.array(X_train)
Y_train = np.array(Y_train)

X_train,Y_train = shuffle(X_train,Y_train,random_state=101)
X_train.shape

X_train,X_test,y_train,y_test = train_test_split(X_train,Y_train,test_size=0.1,random_state=101)

y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train=y_train_new
y_train = tf.keras.utils.to_categorical(y_train)

y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test=y_test_new
y_test = tf.keras.utils.to_categorical(y_test)

from keras.engine.base_layer import regularizers
model = Sequential()
model.add(Conv2D(32,(3,3),activation = 'relu',input_shape=(150,150,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Dropout(0.3))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(256,activation = 'relu'))
model.add(Dense(256,activation = 'relu'))
#model.add(Dense(1024,activation = 'relu',kernel_regularizer=regularizers.l2(0.8)))
model.add(Dropout(0.5))
model.add(Dense(4,activation='softmax'))

from google.colab import drive
drive.mount('/content/drive')

model.summary()

model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

history = model.fit(X_train,y_train,epochs=25,validation_split=0.1)

import matplotlib.pyplot as plt
import seaborn as sns

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(len(acc))
fig = plt.figure(figsize=(14,7))
plt.plot(epochs,acc,'r',label="Training Accuracy")
plt.plot(epochs,val_acc,'b',label="Validation Accuracy")
plt.legend(loc='upper left')
plt.show()

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))
fig = plt.figure(figsize=(14,7))
plt.plot(epochs,loss,'r',label="Training loss")
plt.plot(epochs,val_loss,'b',label="Validation loss")
plt.legend(loc='upper left')
plt.show()

img = cv2.imread('/content/drive/MyDrive/125-136 (1)/Training/pituitary_tumor/p (107).jpg')
img = cv2.resize(img,(150,150))
img_array = np.array(img)
img_array.shape

img_array = img_array.reshape(1,150,150,3)
img_array.shape

from tensorflow.keras.preprocessing import image
img = image.load_img('/content/drive/MyDrive/125-136 (1)/Training/pituitary_tumor/p (107).jpg')
plt.imshow(img,interpolation='nearest')
plt.show()

a=model.predict(img_array)
indices = a.argmax()
indices

ModuleNotFoundError: No module named 'pandas'

In [11]:
from google.colab import files

# Upload the zip file
uploaded = files.upload()


KeyboardInterrupt: 

In [ ]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/tr_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/te_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

classifier.fit_generator(training_set,
                         steps_per_epoch = 230,
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 23)

# Part 3 - Making new predictions

import numpy as np
from keras.preprocessing import image
address = 'dataset/single_prediction/no/Y12.jpg'
test_image = image.load_img(address, target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'No Tumour Detected'
else:
    prediction = 'Brain Tumour Detected'

  #########################################################################################
#image plotting
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

img= Image.open(address)
plt.imshow(img)


############################################################################################
#PLOTTING CURVE FOR LOSSES AND ACCURACY OF SYSTEM
# -*- coding: utf-8 -*-
losses =[0.4299,
0.2220,
0.0983,
0.0511,
0.0310,
0.0103,
0.0140,
0.0058,
0.0125,
0.0055,
9.1392e-04,
5.3858e-04,
0.0017,
0.0259,
0.0099,
0.0065,
7.4795e-04,
2.4068e-04,
1.5254e-04,
2.1131e-04,
6.8710e-05,
5.1168e-05,
5.8422e-05,
2.3587e-05,
3.3743e-05]
accuracy = [0.8111,0.9106,
0.9649,
0.9846,
0.9899,
0.9978,
0.9965,
0.9984,
0.9959,
0.9985,
1.0000,
1.0000,
0.9993,
0.9911,
0.9971,
0.9984,
1.0000,
1.0000,
1.0000,
1.0000,
1.0000,
1.0000,
1.0000,
1.0000,
1.0000]


import matplotlib.pyplot as plt
plt.plot(losses, color= 'red', label = "LOSSES")
plt.plot(accuracy, color = 'blue', label = "ACCURACY")
plt.xlabel('no. of epochs')
plt.legend()
plt.show()

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# CNN Model
def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Load and preprocess data using ImageDataGenerator
def load_and_preprocess_data(train_dir, test_dir, target_size, batch_size):
    train_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical')

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical')

    return train_generator, test_generator

# Train CNN model
def train_cnn_model(train_generator, test_generator, input_shape, num_classes, epochs):
    model = create_cnn_model(input_shape, num_classes)
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples/train_generator.batch_size,
        epochs=epochs,
        validation_data=test_generator,
        validation_steps=test_generator.samples/test_generator.batch_size)
    return model

# Extract features from CNN model
def extract_cnn_features(model, generator):
    feature_extractor = tf.keras.Model(inputs=model.inputs, outputs=model.layers[-2].output)
    features = feature_extractor.predict(generator)
    return features

# Load and preprocess data
train_dir = 'train_data_dir'
test_dir = 'test_data_dir'
target_size = (150, 150)
batch_size = 32
train_generator, test_generator = load_and_preprocess_data(train_dir, test_dir, target_size, batch_size)

# Train CNN model
input_shape = (target_size[0], target_size[1], 3)
num_classes = len(train_generator.class_indices)
epochs = 10
cnn_model = train_cnn_model(train_generator, test_generator, input_shape, num_classes, epochs)

# Extract features using CNN model
train_features = extract_cnn_features(cnn_model, train_generator)
test_features = extract_cnn_features(cnn_model, test_generator)

# Train Random Forest model on CNN features
rf_model = RandomForestClassifier()
rf_model.fit(train_features, train_generator.labels)

# Predict using Random Forest model
rf_predictions = rf_model.predict(test_features)
rf_accuracy = accuracy_score(test_generator.labels, rf_predictions)
print("Random Forest Accuracy:", rf_accuracy)

# Train Gradient Boosting model on CNN features
gb_model = GradientBoostingClassifier()
gb_model.fit(train_features, train_generator.labels)

# Predict using Gradient Boosting model
gb_predictions = gb_model.predict(test_features)
gb_accuracy = accuracy_score(test_generator.labels, gb_predictions)
print("Gradient Boosting Accuracy:", gb_accuracy)


FileNotFoundError: [Errno 2] No such file or directory: 'train_data_dir'